<a href="https://colab.research.google.com/github/DrabberSP/Actividad-4/blob/main/Act4_Regresiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Cargar archivo csv desde equipo
from google.colab import files
files.upload()

In [3]:
# Cargar archivo .xlsx desde equipo
MRetailer=pd.read_excel('microretailer_mit_lift_lab_actualizado.xlsx')

In [4]:
# Obtenemos información del archivo
MRetailer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Columns: 106 entries, _record_id to 235_email
dtypes: float64(20), int64(2), object(84)
memory usage: 273.4+ KB


In [5]:
# Filtro por columnas no consecutivas
#filtro=MRetailer.iloc[:,[13,14,16,21,22,29,31,33,41,42,43,44,45,49,51,56,57,58,59,60,78,90]] #todo el archivo

In [23]:
# Identificar valores nulos por columna
valores_nulos=MRetailer.isnull().sum()
valores_nulos

_record_id                                0
_title                                    0
_server_updated_at                        0
_updated_by                               0
_geometry                                 0
                                       ... 
234_willingness_participate_workshop    125
318_shopkeeper_name                     249
319_shopkeepers_last_name               252
236_whatsapp                            255
235_email                               283
Length: 106, dtype: int64

Reemplazamos Valores Nulos

In [ ]:
# Realizamos una copia del dataframe
Retailers_Nulos=MRetailer.copy()
# Reemplazamos valores nulos
Retailers_Nulos=Retailers_Nulos.fillna(method='bfill')
Retailers_Nulos

In [25]:
# Identificar valores nulos de todo el dataframe
valores_nulos=Retailers_Nulos.isnull().sum().sum()
valores_nulos

1583

In [ ]:
# Copia del dataframe para elimnar más valores nulos
Retailers_N1=Retailers_Nulos.copy()
Retailers_N1=Retailers_Nulos.fillna(method='ffill')
Retailers_N1

In [27]:
# Identificar valores nulos de todo el dataframe
valores_nulos=Retailers_N1.isnull().sum().sum()
valores_nulos

0

Reemplazamos Outliers

In [28]:
R_Filtrado=Retailers_N1.copy()

In [ ]:
#Identificamos las variables no numéricas para eliminar columnas incecesarias de texto
R_Filtrado.info(106)

In [30]:
# Filtro para la columna de numéricos
filtro1=Retailers_N1.iloc[:,[13,14,16,21,22,29,31,33,41,42,43,44,45,49,51,56,57,58,59,60,78,90]]
    #1,2,3,4,5,6,7,8,9,10,11,12,15,17,18,19,20,23,24,25,26,27,28,30,32,34,35,36,37,38,39,40,46,47,48,50,52,53,54,55,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89]]

In [ ]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=filtro1

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
print("Limite superior permitido", Limite_Superior_iqr)
print("Limite inferior permitido", Limite_Inferior_iqr)

In [ ]:
# Encontramos Outliers del Dataframe
outliers_iqr= Retailers_N1[(y>Limite_Superior_iqr)|(y<Limite_Inferior_iqr)]
outliers_iqr

In [ ]:
# Obtenemos datos limpios del Dataframe
data3_iqr= Retailers_N1[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
data3_iqr

In [ ]:
# Corroboramos valores nulos del dataframe LIMPIO
valores_nulos=data3_iqr.isnull().sum()
valores_nulos

In [ ]:
# Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
data4_iqr=data3_iqr.copy()
data4_iqr=data4_iqr.fillna(round(data3_iqr.mean(),1))
data4_iqr

In [ ]:
# Unimos las columnas
Union=Retailers_N1
Datos_limpios=pd.concat([Union, filtro1],axis=1)
Datos_limpios#concatenar con columna limpia de nulos

In [44]:
# Corroboramos valores nulos del dataframe LIMPIO
valores_nulos=Datos_limpios.isnull().sum()
valores_nulos

_record_id                             0
_title                                 0
_server_updated_at                     0
_updated_by                            0
_geometry                              0
                                      ..
317_home_deliveries                    0
62_frequency_only_services             0
78_number_home_deliveries_week         0
172_supplier_frequency                 0
163_number_high_perishable_products    0
Length: 128, dtype: int64

In [45]:
# Convertir archivo a csv
Datos_limpios.to_csv("MicroRetailer_Final.csv")

In [47]:
# Descargar archivo en csv
from google.colab import files
files.download("MicroRetailer_Final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Casos de Regresión lineal y logística

In [ ]:
# Cargar archivo csv desde equipo
from google.colab import files
files.upload()

In [ ]:
# Cargar archivo .xlsx desde equipo
df=pd.read_csv('MicroRetailer_Final.csv')
df.head(5)

Regresión lineal 1

In [ ]:
# Se grafican mapas de dispersión
from turtle import color
sns.scatterplot(x='4_number_permanent_employees_last_year', y='2_current_permanent_employees', color='blue', data=df)
sns.scatterplot(x='145_number_direct_competitors', y='2_current_permanent_employees', color='red', data=df)
sns.scatterplot(x='24_burnout', y='2_current_permanent_employees', color='yellow', data=df)

In [62]:
# Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indep1=df[['4_number_permanent_employees_last_year','145_number_direct_competitors','24_burnout']]
Var_Dep1=df['2_current_permanent_employees']

In [63]:
# Se define model como la función de regresión lineal
from sklearn.linear_model import LinearRegression
model= LinearRegression()

In [64]:
# Verificamos la función relacionada al modelo
type(model)

sklearn.linear_model._base.LinearRegression

In [65]:
# Ajustamos el modelo con las variables antes declaradas
model.fit(X=Vars_Indep1, y=Var_Dep1)

LinearRegression()

In [66]:
# Verificamos los coeficientes obtenidos para el modelo ajustado
model.__dict__

{'fit_intercept': True,
 'normalize': 'deprecated',
 'copy_X': True,
 'n_jobs': None,
 'positive': False,
 'feature_names_in_': array(['4_number_permanent_employees_last_year',
        '145_number_direct_competitors', '24_burnout'], dtype=object),
 'n_features_in_': 3,
 'coef_': array([0.50457784, 0.03992507, 0.06387438]),
 '_residues': 1749.2528405161174,
 'rank_': 3,
 'singular_': array([81.79606507, 76.41957708, 36.11786902]),
 'intercept_': 0.4932986612427839}

In [67]:
# Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R2 de determinación
model.score(Vars_Indep1,Var_Dep1)

0.4943913135058958

Regresión Logística 1

In [68]:
# Redefinimos las variables
X=Vars_Indep1
y=Var_Dep1

In [69]:
# Dividimos el conjunto de datos en la parte de entrenamiento y prueba
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=None)

In [70]:
# Se escalan todos los datos
escalar = StandardScaler()

In [71]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [72]:
# Verificamos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [73]:
# Entrenamos el modelo
algoritmo.fit(X_train , y_train)

LogisticRegression()

In [ ]:
# Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

In [88]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="weighted", pos_label="yes")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.513718747761301


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1375: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5252525252525253


In [90]:
# Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="weighted", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5252525252525253


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1375: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  UserWarning,
